# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Обзор данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.**



In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Обработаем значения в столбце `days_employed`: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
#Доля должников в каждой категории (сумма должников на общее количество людей в каждой категории)
payment = data.groupby('children')['debt'].sum()/data.groupby('children')['debt'].count()

#Метрики для каждой из категорий: общее кол-во людей, кол-во должников.
total_payment = data.groupby('children').agg({'debt': ['count','sum']})

#Строим таблийу и добавляем столбец с процентами.
total_payment['share_%'] = total_payment[('debt', 'sum')]/total_payment[('debt', 'count')]
total_payment['share_%'] = total_payment['share_%'].apply(lambda x: "{0:.2%}".format(x))
print(total_payment)

           debt       share_%
          count   sum        
children                     
0         14091  1063   7.54%
1          4808   444   9.23%
2          2052   194   9.45%
3           330    27   8.18%
4            41     4   9.76%
5             9     0   0.00%


**Вывод:** 

Распределение должников в зависимости от количества детей более-менее равномерно, говорить о какой-то прямой зависимости выплаты долгов и количества детей на основании этих данных нельзя. 

Единственное, можно заметить, что в целом обратившихся за кредитами бездетных людей в разы больше (14091), чем тех, у кого есть хотя бы один ребенок, и процент должников среди бездетных меньше (7,54%), чем у тех, у кого есть хотя бы один ребенок.

За исключением родителей пятерых - но здесь слишком малое количество обращений, всего 9 человек.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#Доля должников для каждого статуса отношений (сумма должников на общее количество людей в каждом статусе)
relationships = data.groupby('family_status')['debt'].sum()/data.groupby('family_status')['debt'].count()

#Метрики для каждого статуса: общее кол-во людей, кол-во должников.
total_relationships = data.groupby('family_status').agg({'debt': ['count','sum']})

#Строим таблийу и добавляем столбец с процентами:
total_relationships['share_%'] = total_relationships[('debt', 'sum')]/total_relationships[('debt', 'count')]
total_relationships['share_%'] = total_relationships['share_%'].apply(lambda x: "{0:.2%}".format(x))
print(total_relationships)


                        debt      share_%
                       count  sum        
family_status                            
Не женат / не замужем   2796  273   9.76%
в разводе               1189   84   7.06%
вдовец / вдова           951   63   6.62%
гражданский брак        4134  385   9.31%
женат / замужем        12261  927   7.56%


**Вывод:** 

Среди клиентов, *не состоящих в браке* или *состоящих в гражданском браке*, доля должников выше (9.76% и 9,31% соотвественно).

*Меньше всего* должников среди вдов/вдовцов (6,62%).
 
Также отметим, что *женатых/замужних клпентов* значительно больше (12261), чем всех остальных. Т.е. либо растущие потребности семьи вынуждают брать кредит, либо наоборот, ощущение, что в семье ты не один сможешь оплачивать ежемесячные взносы, способствует взятию кредита на исполнение своих пожеланий (авто, образование, улучшение жилищных условий). Но данную гипотезу необходимо проверять.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#Категория Е <= 30000 , D <= 50000, С <= 200000, B <= 1000000,  A > 1000000.
#Доля должников для каждого уровня дохода (сумма должников на общее количество людей каждого уровня дохода)
income_level = data.groupby('total_income_category')['debt'].sum()/data.groupby('total_income_category')['debt'].count()

#Метрики для каждого уровня дохода: общее кол-во людей, кол-во должников:
total_income_level = data.groupby('total_income_category').agg({'debt': ['count','sum']})

#Строим таблийу и добавляем столбец с процентами.
total_income_level['share_%'] = total_income_level[('debt', 'sum')]/total_income_level[('debt', 'count')]
total_income_level['share_%'] = total_income_level['share_%'].apply(lambda x: "{0:.2%}".format(x))
print(total_income_level)


                        debt       share_%
                       count   sum        
total_income_category                     
A                         25     2   8.00%
B                       5014   354   7.06%
C                      15921  1353   8.50%
D                        349    21   6.02%
E                         22     2   9.09%


**Вывод:** 

Распределение должников в зависимости от уровня дохода более-менее равномерно.

Единственное, клиенты с *наименьшим доходом* (категория Е, менее 30 тыс. рублей) становятся должниками чуть чаще остальных, в 9,09% случаев. 

*Реже всего* (6,02%) задерживают платежи клиенты с *уровнем дохода D* (от 30 до 50 тыс. рублей).

Также можно отметить, что *больше всего клиентов* (15921 человек), которые брали кредит, *с уровнем дохода C* (от 50 до 200 тыс. руб), *а меньше всего*, 22 человек, *с уровнем дохода E* (менее 30 тыс. руб).

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
#Категория Е <= 30000 , D <= 50000, С <= 200000, B <= 1000000,  A > 1000000.
#Доля должников для каждого уровня дохода (сумма должников на общее количество людей каждого уровня дохода)
purpose_level = data.groupby('purpose_category')['debt'].sum()/data.groupby('purpose_category')['debt'].count()

#Метрики для каждого уровня дохода: общее кол-во людей, кол-во должников:
total_purpose_level = data.groupby('purpose_category').agg({'debt': ['count','sum']})

#Строим таблийу и добавляем столбец с процентами.
total_purpose_level['share_%'] = total_purpose_level[('debt', 'sum')]/total_purpose_level[('debt', 'count')]
total_purpose_level['share_%'] = total_purpose_level['share_%'].apply(lambda x: "{0:.2%}".format(x))
print(total_purpose_level)

                           debt      share_%
                          count  sum        
purpose_category                            
операции с автомобилем     4279  400   9.35%
операции с недвижимостью  10751  780   7.26%
получение образования      3988  369   9.25%
проведение свадьбы         2313  183   7.91%


**Вывод:** 

Исходя из данных, можно сделать вывод, что **ипотечные кредиты** стараются возвращать вовремя чаще - должников всего 7,26%. 
При этом доля ипотечных кредитов среди всех выданных кредитов максимальна - это самая частая цель займа.

**Кредит на проведение свадьбы** также имеет сравнительно неплохой показатель - должников только 7,91%. 
И такие кредиты берут реже по сравнению с остальными.

Самый большой процент должников в категории **Операции с автомобилем** - 9,35%. 

Следует помнить о том, что размеры кредитов сильно отличаются (ипотека может быть и 10 млн, а кредит на свадьбу может быть лишь 200 тыс.).
Вернуть 10 млн куда сложнее, чем 200 тыс. Срок возврата также может быть очень разным (например, 20 лет против 3-5 лет).
Тем не менее, столь глубокое погружение в тему и такая многофакторная модель от нас на данном этапе не требуется.


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

Есть 3 механизма формирования пропусков: **MCAR (Missing Completely At Random), MAR (Missing At Random), MNAR (Missing Not At Random)**. 

*Примером MCAR* можно назвать в нашем случае такую ситуацию: при выгрузке данных часть могла быть утеряна или неверно обработана и выдала ошибку. Или при заполнении анкеты, распечатанной на листе в двухстороннем формате или не слишком удачно сверстанной онлайн, клиент может поспешить и случайно пропустить часть вопросов.

*Примером MAR* в нашем случае могла бы быть следующая ситуация: молодому человеку до 25 лет, который ответил, что он не замужем / не женат, могут не задать вопрос про наличие детей и поставить автоматом 0.

*Примером MNAR* может быть ситуация, когда человек сознательно занижает уровень своих доходов, по разным причинам не желая делиться с банком достоверной и полной информацией. 

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;"> <b>Спасибо за комментарий! </b>
<br>   
В нашем случае есть любопытный момент - пропущенные значения имеются в двух столбцах: <br>
1. <i> days_employed </i> (общий трудовой стаж в днях). <br>
2. <i>total_income</i> (ежемесячный доход). <br>
Причем они встречаются в одних и тех же строках, т.е. если у клиента пропущено значение days_employed, то у него также нет значения total_income. Явной связи с другими столбцами нет (например, пропущено только у пенсионеров - нет такой взаимосвязи).

 Клиенты, конечно, могли сознательно не предоставить информацию о своем доходе в total_income, но тогда им не предоставили бы кредит. Поэтому склоняюсь к общей для столбцов total_income и days_employed <b> ошибке при выгрузке данных.</b>
</div>    

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

Поскольку с количественными переменными мы проводим простые математические операции, нам, во-первых, лучше иметь медианное значение, чем пропуск или ошибку, так как они влияют на окончательный результат и могут в принципе мешать получить его. А во-вторых, при сильных разбросах значений (выбросах) такая значительная разница может увести нас в совсем не ту сторону (при поиске среднего арифметического, медианы и т.д.).

### Шаг 4: общий вывод.

Напишите ваш общий вывод.

#### Резюмируя вышеизложенное:
    * Распределение должников в зависимости от количества детей более-менее равномерно, говорить о какой-то прямой зависимости выплаты долгов и количества детей на основании этих данных нельзя. 
    * В целом обратившихся за кредитами бездетных людей в разы больше (14091), чем тех, у кого есть хотя бы один ребенок, и процент должников среди бездетных меньше (7,54%), чем у тех, у кого есть хотя бы один ребенок.
    * Среди клиентов, не состоящих в браке или состоящих в гражданском браке, доля должников выше (9.76% и 9,31% соотвественно).
    * Меньше всего должников среди вдов/вдовцов (6,62%). А женатых/замужних клпентов значительно больше (12261), чем всех остальных.
    * Распределение должников в зависимости от уровня дохода более-менее равномерно. Клиенты с наименьшим доходом (категория Е, менее 30 тыс. рублей) становятся должниками чуть чаще остальных, в 9,09% случаев. 
    * Реже всего (6,02%) задерживают платежи клиенты с уровнем дохода D (от 30 до 50 тыс. рублей).
    * Больше всего клиентов (15921 человек), которые брали кредит, с уровнем дохода C (от 50 до 200 тыс. руб), а меньше всего, 22 человек, с уровнем дохода E (менее 30 тыс. руб).  
    * Ипотеку чаще стараются возвращать вовремя - должников всего 7,26%. При этом доля ипотечных кредитов среди всех выданных кредитов максимальна - это самая популярная цель займа.
    * По кредиту на проведение свадьбы должников только 7,91%. И такие кредиты берут реже по сравнению с остальными.
    * Самый большой процент должников в категории "Операции с автомобилем" - 9,35%. 
        